In [2]:
import pathlib
from datetime import datetime
from typing import List, Tuple, Union, Dict

import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import Column

from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import Window


In [3]:
# master configuration to use only 4 CPU cores
spark = SparkSession.builder.master("local[4]").getOrCreate()

# basic configuration to use only a reasonable number of partitions
spark.conf.set("spark.sql.shuffle.partition", 4)

# configuration to work in UTC
spark.conf.set("spark.sql.session.timeZone", "UTC")


23/06/23 14:10:09 WARN Utils: Your hostname, Emilianos-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.87 instead (on interface en0)
23/06/23 14:10:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/23 14:10:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Read JSONL file
df = (
    spark
    .read
    # .option("multiline", "true")
    .json("/Users/emilianofrigo/Downloads/1687510071631_0.json")
)

23/06/23 14:11:38 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
df.printSchema()

root
 |-- _airbyte_ab_id: string (nullable = true)
 |-- _airbyte_data: struct (nullable = true)
 |    |-- AMS Numbers: string (nullable = true)
 |    |-- Actual Arrival Port Dwell Time: string (nullable = true)
 |    |-- Actual Arrival Port to Delivery Transit Time: string (nullable = true)
 |    |-- Actual CRD to Pickup Dwell Time: string (nullable = true)
 |    |-- Actual Departure Port Dwell Time: string (nullable = true)
 |    |-- Actual Departure Port to Arrival Port Transit Time: string (nullable = true)
 |    |-- Actual Departure Port to Discharge Port Transit Time: string (nullable = true)
 |    |-- Actual Discharge Port Dwell Time: string (nullable = true)
 |    |-- Actual Discharge Port to Arrival Port Transit Time: string (nullable = true)
 |    |-- Actual Door to Door Transit Time: string (nullable = true)
 |    |-- Actual Flexport-Responsible Transit Time: string (nullable = true)
 |    |-- Actual Pickup to Departure Port Transit Time: string (nullable = true)
 |    |-- Ac

In [6]:
df.columns

['_airbyte_ab_id', '_airbyte_data', '_airbyte_emitted_at']

In [7]:
df.show(
    2,
    vertical=True
)

-RECORD 0-----------------------------------
 _airbyte_ab_id      | cccc45c5-2cab-42a... 
 _airbyte_data       | {FLXT00001991629A... 
 _airbyte_emitted_at | 1687510074000        
-RECORD 1-----------------------------------
 _airbyte_ab_id      | 504b6864-f7cf-454... 
 _airbyte_data       | {FLXT00001204241A... 
 _airbyte_emitted_at | 1687510074000        
only showing top 2 rows



In [8]:

check_df = spark.read.parquet("/Users/emilianofrigo/Downloads/part-00000-661bb95e-f427-47b9-83d5-6fdda8f77e97.c000.snappy.parquet")


In [9]:
check_df.printSchema()

root
 |-- report_ingested_ts: timestamp (nullable = true)
 |-- raw_data: struct (nullable = true)
 |    |-- AMS_Numbers: string (nullable = true)
 |    |-- Actual_Arrival_Port_Dwell_Time: string (nullable = true)
 |    |-- Actual_Arrival_Port_to_Delivery_Transit_Time: string (nullable = true)
 |    |-- Actual_CRD_to_Pickup_Dwell_Time: string (nullable = true)
 |    |-- Actual_Departure_Port_Dwell_Time: string (nullable = true)
 |    |-- Actual_Departure_Port_to_Arrival_Port_Transit_Time: string (nullable = true)
 |    |-- Actual_Departure_Port_to_Discharge_Port_Transit_Time: string (nullable = true)
 |    |-- Actual_Discharge_Port_Dwell_Time: string (nullable = true)
 |    |-- Actual_Discharge_Port_to_Arrival_Port_Transit_Time: string (nullable = true)
 |    |-- Actual_Door_to_Door_Transit_Time: string (nullable = true)
 |    |-- Actual_Flexport-Responsible_Transit_Time: string (nullable = true)
 |    |-- Actual_Pickup_to_Departure_Port_Transit_Time: string (nullable = true)
 |    |-- 

In [10]:
def json_writer(
    df: DataFrame,
    path: str,
    mode: str = "overwrite",
    partition_cols: List[str] = None,
) -> None:
    """Writes DataFrame in json format for a specified path.

    Parameters
    ----------
    df : DataFrame
        DataFrame needed to be written as json files.
    path : str
        Desired target path to write json files to.
    mode : str, optional
        Desired write mode, by default "overwrite".
    partition_cols : List[str], optional
        Desired partition scheme, by default None.
    """

    partition_cols = partition_cols or []

    (df.write.mode(mode).partitionBy(partition_cols).json(path))

In [11]:
json_writer(
            df=check_df,
            path="/Users/emilianofrigo/Downloads/output.jsonl",
            # partition_cols=[
            #     "p_brand_id",
            #     "p_region_id",
            #     "p_entity_id",
            #     "p_ingestion_dt",
            # ],
        )

23/06/24 00:11:53 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 354015 ms exceeds timeout 120000 ms
23/06/24 00:11:53 WARN SparkContext: Killing executors is not supported by current scheduler.
23/06/24 00:11:59 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:103)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:87)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:643)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1057)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:238)
	at sc